In [1]:
# !pip install finance-datareader

In [7]:
import pymysql
from sqlalchemy import create_engine, text
from dotenv import load_dotenv
import os
import pandas as pd
from datetime import datetime

# .env 파일에서 환경 변수 로드
load_dotenv()

# MySQL 연결 정보
db_user = os.getenv('DB_USER')
db_password = os.getenv('DB_PASSWORD')
db_host = os.getenv('DB_HOST')
db_name = os.getenv('DB_NAME')

# 데이터베이스 생성 함수
def create_database_if_not_exists():
    # MySQL에 접속
    connection = pymysql.connect(
        host=db_host,
        user=db_user,
        password=db_password
    )
    
    try:
        with connection.cursor() as cursor:
            # 데이터베이스 존재 여부 확인 및 없을 경우 생성
            cursor.execute(f"CREATE DATABASE IF NOT EXISTS {db_name};")
            print(f"Database '{db_name}' checked/created successfully.")
    finally:
        connection.close()

# SQLAlchemy 엔진 생성
engine = create_engine(f"mysql+pymysql://{db_user}:{db_password}@{db_host}/{db_name}")

# 테이블 존재 여부 확인 함수
def check_table_exists(table_name):
    with engine.connect() as conn:
        query = text(f"SHOW TABLES LIKE '{table_name}'")
        result = conn.execute(query).fetchall()
        return len(result) > 0

# 한국 주식 목록 가져오기 (KRX)
def get_kr_stock_list():
    url = 'http://kind.krx.co.kr/corpgeneral/corpList.do?method=download'
    
    # 인코딩을 'euc-kr'로 설정하여 읽기
    kr_stock_list = pd.read_html(url, header=0, encoding='euc-kr')[0]
    
    # 종목코드를 6자리로 맞춤
    kr_stock_list['종목코드'] = kr_stock_list['종목코드'].apply(lambda x: f'{x:06d}')
    
    return kr_stock_list

# 상장 기업 목록을 DB에 저장하는 함수
def insert_stock_company_data():
    # 한국 상장 기업 목록 가져오기
    stock_list = get_kr_stock_list()
    
    # 필요에 따라 열 이름을 맞춤 (원하는 열로 수정 가능)
    # stock_list = stock_list[['종목코드', '회사명', '시장구분', '상장주식수', '자본금']]

    # 데이터프레임을 MySQL에 저장
    stock_list.to_sql('korea_stock_company', con=engine, if_exists='replace', index=False)
    print("Stock company data inserted successfully.")

# 함수 실행 부분
if __name__ == "__main__":
    # 데이터베이스 생성 확인
    create_database_if_not_exists()
    
    # 상장 기업 데이터 삽입
    if not check_table_exists('korea_stock_company'):
        insert_stock_company_data()
    else:
        print("Table 'korea_stock_company' already exists.")


Database 'korea_stock' checked/created successfully.


C:\Users\5-29\miniforge3\envs\fintech\lib\site-packages\bs4\__init__.py:228: UserWarning: You provided Unicode markup but also provided a value for from_encoding. Your from_encoding will be ignored.
  warnings.warn("You provided Unicode markup but also provided a value for from_encoding. Your from_encoding will be ignored.")


Table 'korea_stock_company' created successfully based on dataframe columns.
New stock company data inserted successfully.
